In [1]:
import race_results_scanner
import google_cloud_downloader as gcd
from PyPDF2 import PdfReader
import numpy as np
import pandas as pd
import os
import header_scanner as hs


pdf = "Error_PDFS/AQU_11_04_2022.pdf"
#pdf = "Example_PDFS/equibaseFile.pdf"


In [2]:
#Testing for page

page_list = race_results_scanner.get_page_list(pdf)
target_page = 10

page_list_index = 0
for page in page_list:
    if(target_page == page['page_num']):
        break
    page_list_index += 1
    
reader = PdfReader(pdf) #File to be scanned
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()

page = page_list[page_list_index]
header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
for field,value in header.items():
    print(field ,':', value)
result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan
print(result_tables[0])
print(result_tables[1])

The PDF <_io.BufferedReader name='Error_PDFS/AQU_11_04_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


track : Aqueduct
date : November 4, 2022
race_num : 9
criteria : FOR FILLIES AND MARES THREE YEARS OLD AND UPWARD
track_length :  One And One Eighth Miles On The Dirt
purse : $150,000
weather : Clear
track_type : Fast
off_time : 4:45
start : Good for all except 1
race_type : stakes
Looking for tables...
Found table top bound at 162.24599999999998
Found table bottom bound at 233.54599999999994
Looking for tables...
Found table top bound at 427.871
Found table bottom bound at 512.565
    Last Raced  Pgm jockey_first_name jockey_last_name   PP  Start  1/4  1/2  \
0   1Oct22 LRL  7.0          Kendrick        Carmouche  5.0    1.0    1    1   
1  17Sep22 BAQ  4.0            Hector        Diaz  Jr.  3.0    4.0    4    4   
2   9Oct22 BAQ  3.0            Javier       Castellano  2.0    2.0    3    3   
3   1Sep22 SAR  6.0            Trevor         McCarthy  4.0    3.0    2    2   
4  19Aug22 SAR  1.0           Michael            Luzzi  1.0    5.0  ---  ---   

   3/4  Str  ...             Com

In [3]:
result_tables[1]

,Pgm,Horse Name,Start,1/4,1/2,3/4,Str,Fin,1/4_length_behind,1/2_length_behind,3/4_length_behind,Str_length_behind,Fin_length_behind
0,7,Battle Bling,1.0,1,1,1,1,1,1,1/2,1,3 1/2,Nec
1,4,Nostalgic,4.0,4,4,3,2,2,3,2,2,3 1/2,Nec
2,3,First to Act,2.0,3,3,4,3,3,2,1 1/2,2 1/2,8 1/2,14
3,6,W W Fitzy,3.0,2,2,2,4,4,1,1/2,1,15 1/2,33 1/
4,1,Exotic West,5.0,---,---,---,---,---,NaN,NaN,NaN,NaN,NaN


In [2]:
#Testing for whole pdf
master_df = pd.DataFrame()
page_list = race_results_scanner.get_page_list(pdf)
print(page_list)
    
reader = PdfReader(pdf) #File to be scanned
number_of_pages = len(reader.pages) #Number of pages
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()
i = 0
for page in page_list:
    header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
    print(header)
    result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan
                
    #Combine into page DF
    top_table = result_tables[0]
    bottom_table = result_tables[1]
    #Dropping horse name for merge
    bottom_table = bottom_table.drop("Horse Name", axis = 1).astype(object)
    merged_df = top_table.join(bottom_table.set_index("Pgm"), on = "Pgm", rsuffix = "_RLP")
    for field,value in header.items():
        merged_df[field] = value
        #Changing all cols to object type 
        merged_df = merged_df.astype(object)
        #For every page df
    #Merge into master pdf df
    if(len(pdf_df) < 1):
        pdf_df = merged_df
    else:
        pdf_df = pd.merge(pdf_df, merged_df, how = 'outer')
            
    print(pdf_df)
    #Merging into master 
    #if(len(master_df) < 1):
     #   master_df = pdf_df
    #else:
    #    master_df = pd.merge(master_df,pdf_df, how = 'outer')

The PDF <_io.BufferedReader name='Error_PDFS/AQU_11_04_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


[{'page_num': 0, 'horse_count': 7}, {'page_num': 1, 'horse_count': 6}, {'page_num': 2, 'horse_count': 7}, {'page_num': 3, 'horse_count': 7}, {'page_num': 4, 'horse_count': 9}, {'page_num': 6, 'horse_count': 8}, {'page_num': 7, 'horse_count': 11}, {'page_num': 9, 'horse_count': 7}, {'page_num': 10, 'horse_count': 5}, {'page_num': 11, 'horse_count': 9}]
{'track': 'Aqueduct', 'date': 'November 4, 2022', 'race_num': '1', 'criteria': '(UP TO $14,790 NYSBFOA) FOR MAIDENS, FILLIES TWO YEARS OLD', 'track_length': ' One Mile On The Dirt', 'purse': '$85,000', 'weather': 'Clear', 'track_type': 'Fast', 'off_time': '11:38', 'start': 'Good for all except 2,6', 'race_type': 'maiden special weight'}
Looking for tables...
Found table top bound at 125.82999999999993
Found table bottom bound at 207.13
Looking for tables...
Found table top bound at 351.955


The PDF <_io.BufferedReader name='Error_PDFS/AQU_11_04_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Found table bottom bound at 445.233
   Last Raced  Pgm jockey_first_name  jockey_last_name   PP Start 1/4 1/2 3/4  \
0  2Oct22 BAQ  7.0            Javier        Castellano  7.0   2.0   2   2   1   
1         ---  1.0              Eric            Cancel  1.0   4.0   4   4   3   
2         ---  4.0            Trevor          McCarthy  4.0   6.0   7   6   4   
3  4Sep22 SAR  3.0          Kendrick         Carmouche  3.0   3.0   1   1   2   
4         ---  5.0            Hector         Diaz  Jr.  5.0   5.0   6   7   6   
5         ---  6.0              Omar  Hernandez Moreno  6.0   1.0   3   3   5   
6         ---  2.0              Jose             Gomez  2.0   7.0   5   5   7   

  Str  ...              date race_num  \
0   1  ...  November 4, 2022        1   
1   2  ...  November 4, 2022        1   
2   4  ...  November 4, 2022        1   
3   3  ...  November 4, 2022        1   
4   6  ...  November 4, 2022        1   
5   5  ...  November 4, 2022        1   
6   7  ...  November 4, 2022

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
The PDF <_io.BufferedReader name='Error_PDFS/AQU_11_04_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


...

Found table bottom bound at 595.817
     Last Raced   Pgm jockey_first_name  jockey_last_name    PP Start 1/4 1/2  \
0    2Oct22 BAQ   7.0            Javier        Castellano   7.0   2.0   2   2   
1           ---   1.0              Eric            Cancel   1.0   4.0   4   4   
2           ---   4.0            Trevor          McCarthy   4.0   6.0   7   6   
3    4Sep22 SAR   3.0          Kendrick         Carmouche   3.0   3.0   1   1   
4           ---   5.0            Hector         Diaz  Jr.   5.0   5.0   6   7   
5           ---   6.0              Omar  Hernandez Moreno   6.0   1.0   3   3   
6           ---   2.0              Jose             Gomez   2.0   7.0   5   5   
7    6Oct22 BAQ   6.0             Jorge       Vargas  Jr.   6.0   1.0   3   2   
8    6Oct22 BAQ   4.0            Javier        Castellano   4.0   4.0   4   4   
9   15Oct22 BAQ   1.0            Hector         Diaz  Jr.   1.0   2.0   2   3   
10  30Sep22 BAQ   5.0            Trevor          McCarthy   5.0   5.0   6

Looking for tables...
Found table top bound at 143.82999999999993
Found table bottom bound at 225.13
Looking for tables...
Found table top bound at 385.705


C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
The PDF <_io.BufferedReader name='Error_PDFS/AQU_11_04_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Found table bottom bound at 478.983
    Last Raced  Pgm jockey_first_name  jockey_last_name   PP Start 1/4 1/2  \
0   2Oct22 BAQ  7.0            Javier        Castellano  7.0   2.0   2   2   
1          ---  1.0              Eric            Cancel  1.0   4.0   4   4   
2          ---  4.0            Trevor          McCarthy  4.0   6.0   7   6   
3   4Sep22 SAR  3.0          Kendrick         Carmouche  3.0   3.0   1   1   
4          ---  5.0            Hector         Diaz  Jr.  5.0   5.0   6   7   
..         ...  ...               ...               ...  ...   ...  ..  ..   
57  9Oct22 BAQ  6.0            Trevor          McCarthy  6.0   1.0   3   3   
58  9Oct22 BAQ  2.0            Javier        Castellano  2.0   5.0   4   4   
59  8Oct22 BAQ  4.0          Kendrick         Carmouche  4.0   4.0   5   5   
60  9Oct22 BAQ  3.0              Jose             Gomez  3.0   7.0   7   7   
61  19Oct22 FL  7.0              Luis  Rodriguez Castro  7.0   2.0   2   2   

   3/4 Str  ... race_num   

ValueError: You are trying to merge on float64 and object columns. If you wish to proceed you should use pd.concat

In [ ]:
pdf_df

In [6]:
pdf_df.to_csv("ARAP.csv")